 # Opis notatnika
 W poprzednich krokach pobraliśmy dane oraz przygotowaliśmy bazę `Postgres` na import. Głównym celem w tym notatniku jest  odpowiednie dostosowanie struktury danych z plików źródłowych do formatu zgodnego z `Postgres`, a następnie wgranie ich na nasz serwer. 

In [1]:
#Pobranie wymaganych bibliotek i połączenie z bazą danych:

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

username = 'postgres'
password='al2cx7hsixj',  # Przypadkowe, nieprawdziwe hasło

host = 'localhost'
database = 'airlines'
port = 5432


In [2]:
# Tworzenie zmiennej engine do przesyłania danych.

url = URL.create(
    "postgresql",
    username="postgres",
    password="aaaa", 
    host="localhost",
    database="airlines",
)
engine = create_engine(url)

In [3]:
# Definiowanie funkcji ładującej dane do obszaru roboczego

def load_raw_data(file_name):
    df = pd.read_csv(f"C:\\Coderslab_Analityk_kurs\\Projekt_koncowy\\data\\raw\\{file_name}")
    df.columns = df.columns.str.lower()
    return df

 # Zaczytanie poszczególnych plików do ramek


In [4]:
df_aircraft = load_raw_data('aircraft.csv')
df_airport_list = load_raw_data('airport_list.csv')
df_airport_weather = load_raw_data('airport_weather.csv')
df_flight = load_raw_data('flight.csv')

In [30]:
# Wizualne sprawdzenie, jak wygląda przykładowa ramka

df_aircraft.head()

,manufacture_year,tail_num,number_of_seats
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0


In [5]:
# Zarówno podczas zapisywania i wczytywania została dodana kolumna z indeksami. 
# Usunięcie tej zduplikowanej, nienazwanej kolumny z indeksami:

df_aircraft = df_aircraft.drop('unnamed: 0', axis=1)
df_airport_list = df_airport_list.drop('unnamed: 0', axis=1)
df_airport_weather = df_airport_weather.drop('unnamed: 0', axis=1)
df_flight = df_flight.drop('unnamed: 0', axis=1)

In [6]:
# Definiowanie funkcji eksportującej dane do bazy danych:

def export_table_to_db(df, table_name):
    df.to_sql(table_name, 
                 engine, 
                 schema=None, 
                 if_exists='append', 
                 index=None, 
                 index_label=None, 
                 chunksize=1000, 
                 )

 ## Wgrywanie danych

 ### Wgranie `aircraft_df` do tabeli `aircraft`

In [7]:
export_table_to_db(df_aircraft, 'aircraft')

 ### Wgranie `airport_weather_df` do tabeli `airport_weather`

In [8]:
export_table_to_db(df_airport_weather, 'airport_weather')

 ### Wgranie `flight_df` do tabeli `flight`

In [9]:
export_table_to_db(df_flight, 'flight')

 ### Wgranie `airport_list_df` do tabeli `airport_list`

In [10]:
export_table_to_db(df_airport_list, 'airport_list')

In [13]:
# Sprawdzenie czy dane faktycznie zostały zapisane (poprzez pobranie za pomocą metody cursor):

import psycopg2


con = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='aaaa',
    dbname='airlines', 
)
cursor = con.cursor()

cursor.execute("SELECT * FROM aircraft")

columns = [desc[0] for desc in cursor.description]
data = cursor.fetchall()


# Zamknięcie cusora

con.close()
df_from_db = pd.DataFrame(data, columns=columns)
df_from_db 

,id,manufacture_year,tail_num,number_of_seats
0,1,1944,N54514,0.0
1,2,1945,N1651M,0.0
2,3,1953,N100CE,0.0
3,4,1953,N141FL,0.0
4,5,1953,N151FL,0.0
...,...,...,...,...
7378,7379,2019,N14011,337.0
7379,7380,2019,N16008,337.0
7380,7381,2019,N16009,337.0
7381,7382,2019,N2250U,276.0


 # Podsumowanie
 W tym notatniku załadowaliśmy pobrane wcześniej pliki na bazę danych. Dzięki temu stworzyliśmy centralne miejsce ich magazynowania, co wykorzystamy zarówno przy analizie danych, jak i przy późniejszej budowie systemu raportowego.